# 在智能体运行时中使用 MCP 服务

AgentKit 智能体运行时可以访问 AgentKit 网关部署的 MCP 服务和 MCP 工具集。

在本教程中，我们将演示如何在 AgentKit 智能体运行时中部署智能体，同时访问 AgentKit 网关部署的 MCP 服务和 MCP 工具集，你将学会，

1. 如何使用 AgentKit 智能体运行时中的智能体访问 AgentKit 网关中的 MCP 服务。

本教程将部署一个端到端应用，在开始本教程之前，请确保你已经学习了 AgentKit SDK Gateway 教程的前几节内容。

> **注意**：本教程需要使用火山引擎 SDK。因此，你需要将 `VOLCENGINE_ACCESS_KEY`、`VOLCENGINE_SECRET_KEY`、`VOLCENGINE_REGION` 和 `MODEL_API_KEY` 配置在 `.env` 文件中。

## 1. 在函数应用中部署 API 服务

AgentKit 网关支持将存量 API 服务转换为 MCP 服务。将存量 API 服务转换为 MCP 服务，是一种利旧，是实现经济效益的高效路径，目的是使自身的服务能被外部的智能体调用，从而提升存量服务的价值。以高德地图为例，高德将存量的 API 服务，例如 IP 定位、地理编码转换为 MCP 服务，就可以让外部智能体调用高德地图，从而提升服务的活跃度。

MCP 大幅降低了智能体访问和调用外部资源、数据和服务的复杂度，但如果您将复用存量基础设施作为 MCP 服务的开发策略，那么将面对一个新的问题，从存量 API 服务到 MCP 服务的转换。存量 API 服务转换为 MCP 服务是一个重复的体力活，日常还需要进行各类维护，例如接口更新和稳定性保障等。将存量 API 服务对外通过 MCP 协议暴露成 MCP 服务，这个过程包括：

1. 读取并解析 API 文档，获得访问路径、请求方法、请求参数和响应等。
2. 按照 MCP 协议，将 API 转换为 MCP 工具。
3. 对于 `tools/list` MCP 请求，返回由 API 转换后的 MCP 工具列表。
4. 对于 `tools/call` MCP 请求，解析 MCP 请求中的 JSON-RPC 内容，生成 API 请求发送到后端。当后端响应时，将 API 响应包装成 MCP 响应返回给 MCP 客户端。

接下来，我们先在函数服务中部署天气 Mock 服务。这个天气 Mock 服务总是会告诉我们今天在下雨，气温是 25 度。

In [ ]:
!pip install volcengine-python-sdk --quiet

In [ ]:
import base64
import dotenv
import os
import time
import uuid
import volcenginesdkcore
from volcenginesdkvefaas import (
    VEFAASApi,
    CreateFunctionRequest,
    CreateFunctionResponse,
    ReleaseRequest,
    ReleaseResponse,
    GetReleaseStatusRequest,
    GetReleaseStatusResponse,
)

dotenv.load_dotenv()

id = str(uuid.uuid4())[-8:]

configuration = volcenginesdkcore.Configuration()
configuration.ak = os.getenv("VOLCENGINE_ACCESS_KEY")
configuration.sk = os.getenv("VOLCENGINE_SECRET_KEY")
configuration.region = os.getenv("VOLCENGINE_REGION")
volcenginesdkcore.Configuration.set_default(configuration)

source = b""
with open("resources/volc-weather.zip", "rb") as f:
    content = f.read()
    source = base64.b64encode(content).decode()

api_instance = VEFAASApi()
request = CreateFunctionRequest(
    name=f"function-{id}",
    runtime="native-node20/v1",
    source_type="zip",
    source=source,
    command="./run.sh",
    project_name="default",
)

function_id = ""
try:
    response: CreateFunctionResponse = api_instance.create_function(request)
    function_id = response.id
    print(f"Function created successfully! Function ID: {function_id}")
except Exception as e:
    print(f"An error occurred: {e}")
    raise e

request = ReleaseRequest(
    function_id=function_id,
    revision_number=0,
)

try:
    response: ReleaseResponse = api_instance.release(request)
except Exception as e:
    print(f"An error occurred: {e}")
    raise e


request = GetReleaseStatusRequest(
    function_id=function_id,
)

try:
    while True:
        response: GetReleaseStatusResponse = api_instance.get_release_status(request)
        if response.status == "done":
            break
        if response.status != "inprogress":
            print(f"The function is in {response.status}")
            raise RuntimeError()
        time.sleep(2)
except Exception as e:
    print(f"An error occurred: {e}")
    raise e

## 2. 通过 MCP 服务访问函数应用中的 API 服务

接下来，我们使用 AgentKit 网关将 API 服务转换为 MCP 服务。

In [ ]:
from agentkit.sdk.mcp.client import AgentkitMCPClient
from agentkit.sdk.mcp.types import (
    CreateMCPServiceRequest,
    ProtocolForCreateMCPService,
    ProtocolHttpApiForCreateMCPService,
    NetworkForCreateMCPService,
    InboundAuthorizerForCreateMCPService,
    InboundAuthorizerAuthorizerForCreateMCPService,
    InboundAuthorizerAuthorizerKeyAuthForCreateMCPService,
    InboundAuthorizerAuthorizerKeyAuthApiKeysItemForCreateMCPService,
    BackendForCreateMCPService,
    BackendFunctionForCreateMCPService,
)

id = str(uuid.uuid4())[-8:]

api_config = ""
with open("resources/volc-weather.json", "r") as f:
    api_config = f.read()

client = AgentkitMCPClient()
request = CreateMCPServiceRequest(
    Name=f"mcp-service-{id}",
    # 转换 API 服务到 MCP 服务时，`ProtocolType` 为 `HTTP`。
    ProtocolType="HTTP",
    ProtocolConfiguration=ProtocolForCreateMCPService(
        HttpApiConfiguration=ProtocolHttpApiForCreateMCPService(
            Configuration=f"{base64.b64encode(api_config.encode()).decode()}",
        )
    ),
    Path="/mcp",
    NetworkConfiguration=NetworkForCreateMCPService(
        EnablePublicNetwork=True,
        EnablePrivateNetwork=False,
    ),
    InboundAuthorizerConfiguration=InboundAuthorizerForCreateMCPService(
        AuthorizerType="ApiKey",
        Authorizer=InboundAuthorizerAuthorizerForCreateMCPService(
            KeyAuth=InboundAuthorizerAuthorizerKeyAuthForCreateMCPService(
                ApiKeys=[
                    InboundAuthorizerAuthorizerKeyAuthApiKeysItemForCreateMCPService(
                        Name=f"mcp-service-api-key-{id}",
                    ),
                ],
            ),
        ),
    ),
    BackendType="Function",
    BackendConfiguration=BackendForCreateMCPService(
        FunctionConfiguration=BackendFunctionForCreateMCPService(
            FunctionId=function_id,
        ),
    ),
    ProjectName="default",
)

mcp_service_id = ""
try:
    response = client.create_mcp_service(request)
    mcp_service_id = response.mcp_service_id
    print(f"MCP service created successfully! MCP service ID: {mcp_service_id}")
except Exception as e:
    print(f"An error occurred: {e}")
    raise e

AgentKit Gateway 部署 MCP 服务需要一段时间。在部署完成后，我们可以检查一下 API 服务 转换的 MCP 服务是否可用。

In [ ]:
from agentkit.sdk.mcp.types import (
    GetMCPServiceRequest,
)

request = GetMCPServiceRequest(
    MCPServiceId=mcp_service_id,
)

endpoint = ""
api_key = ""
try:
    while True:
        response = client.get_mcp_service(request)
        if response.mcp_service.status == "Ready":
            endpoint = response.mcp_service.network_configurations[0].endpoint
            api_key = response.mcp_service.inbound_authorizer_configuration.authorizer.key_auth.api_keys[0].key
            break
        if response.mcp_service.status != "Creating":
            print(f"The MCP service is in {response.mcp_service.status}")
            raise RuntimeError()
        time.sleep(2)
except Exception as e:
    print(f"An error occurred: {e}")
    raise e

url = f"{endpoint}/mcp"
print(f"MCP service URL: {url}")
print(
    f'MCP service API key: {api_key}\n  (Hint: Add a "Authorization: Bearer {api_key}" request header to access the MCP service)'
)

In [ ]:
!pip install fastmcp --quiet

In [ ]:
from fastmcp import Client
from fastmcp.client.transports import StreamableHttpTransport

transport = StreamableHttpTransport(
    url=url,
    headers={
        "Authorization": f"Bearer {api_key}",
    }
)
mcp_client = Client(transport)

async with mcp_client:
    tools = await mcp_client.list_tools()
    print(f"The MCP server has {len(tools)} tools.")
    for tool in tools:
        print(f"- {tool.name}")

## 3. 在智能体运行时中使用 MCP 服务

我们的天气 Mock MCP 服务已经部署完成了，可以看到，这个 MCP 服务包含了 3 个工具：查询天气情况、查询降水情况以及查询气象预警，看着不错！

我们准备一下部署到 AgentKit 智能体运行时的智能体。关于 AgentKit 智能体运行时，你可以阅读 AgentKit SDK Runtime 教程。

In [ ]:
%%writefile my_agent.py
from agentkit.apps import AgentkitSimpleApp
from google.adk.tools.mcp_tool.mcp_toolset import (
    MCPToolset,
    StreamableHTTPConnectionParams,
)
import os
from veadk import Runner, Agent

url = os.getenv("MCP_SERVICE_URL")
api_key = os.getenv("MCP_SERVICE_API_KEY")
get_city_weather = MCPToolset(
    connection_params=StreamableHTTPConnectionParams(
        url=url,
        headers={"Authorization": f"Bearer {api_key}"},
    ),
)

app = AgentkitSimpleApp()
agent = Agent(model_name="doubao-seed-1-6-250615", tools=[get_city_weather])
runner = Runner(agent=agent)


@app.entrypoint
async def run(payload: dict, headers: dict) -> str:
    prompt = payload["prompt"]
    if prompt is None:
        return "prompt is None"

    response = await runner.run(
        messages=prompt,
    )
    return response


@app.ping
def ping() -> str:
    return "pong!"


if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000)

AgentKit 智能体运行时的 `CreateRuntime` API 支持全面的配置选项。您可以指定容器镜像、环境变量和加密设置，还可以配置协议类型（HTTP、MCP）和入站认证，以控制客户端与智能体的通信方式。

> **注意**：我们建议你将代码打包成容器，结合火山引擎镜像仓库产品，通过 CI/CD 流水线和基础设施即代码（IaC）将镜像推送到镜像仓库中。

在本教程中，我们将使用 AgentKit Python SDK 轻松打包您的代码并将其部署到 AgentKit 智能体运行时中。

在配置步骤中，我们将根据您的应用程序代码自动生成 `agentkit.yaml` 配置文件，它用于在本地保存 AgentKit 智能体运行时的配置。

> **说明**：本教程使用 `cloud` 部署模式，即在远程构建镜像并自动部署到 AgentKit 智能体运行时中。

In [ ]:
from agentkit.toolkit.sdk.config import AgentConfig

MODEL_API_KEY = os.getenv("MODEL_API_KEY")

config = (
    AgentConfig("agentkit.yaml")
    .set("common.agent_name", "http_to_mcp_agent")
    .set("common.entry_point", "my_agent.py")
    .set("common.language", "Python")
    .set("common.language_version", "3.12")
    .set("common.launch_type", "cloud")
    .set("launch_types.cloud.region", "cn-beijing")
    .set("launch_types.cloud.runtime_id", "")
    .set("launch_types.cloud.runtime_name", "")
    .set("launch_types.cloud.runtime_role_name", "")
    .set("launch_types.cloud.runtime_envs.MODEL_AGENT_API_KEY", MODEL_API_KEY)
    .set("launch_types.cloud.runtime_envs.MCP_SERVICE_URL", url)
    .set("launch_types.cloud.runtime_envs.MCP_SERVICE_API_KEY", api_key)
    .save()
)

print("Configuration created and saved!")

我们已经准备好了关于智能体的一切，现在，我们就将它部署到 AgentKit 智能体运行时中。在部署过程中，AgentKit 将，

1. 构建镜像。

   1. 生成 Dockerfile，根据项目配置自动生成容器化配置，
   2. 构建 Docker 镜像，将你的智能体代码打包成容器镜像，
   3. 准备镜像仓库，
   4. 将构建好的镜像推送到云端镜像仓库中。

2. 在 AgentKit 智能体运行时中部署智能体。

   1. 创建 IAM 角色，为 AgentKit 智能体运行时生成必要的权限角色，
   2. 创建 AgentKit 智能体运行时。

In [ ]:
from agentkit.toolkit import sdk

result = sdk.launch()
print(f"Agent deployed successfully. Agent runtime ID: {result.deploy_result.service_id}")

最后，让我们试着调用 AgentKit 智能体运行时中的智能体。

In [ ]:
invoke_response = sdk.invoke(
    payload={
        "prompt": "今天上海的天气怎么样？",
    }
)
print(f"The agent responds with \"{invoke_response.response}\".")

## 总结

恭喜你！现在你已经掌握了在智能体运行时中访问 MCP 服务的方法，

1. 通过 Swagger 或 OpenAPI 3 文档转换函数服务中的 API 服务到 MCP 服务。
2. 创建智能体运行时访问 MCP 服务。

现在，请继续学习其他教程。